In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content['result']

In [3]:
def getSuperCommittees():
    method = "hmy_getSuperCommittees"
    params = []
    return get_information(method, params)['current']

In [4]:
def M5_test():
    flag = True
    # get the median stake and the upper and lower level 
    result = getSuperCommittees()
    median = float(result['epos-median-stake'])
    lower = (median- 0.15*median)
    upper = (median + 0.15*median)
    print("median stake is " + str(median))
    print("lower bond is " + str(lower))
    print("upper bond is " + str(upper))
    
    shards = result['quorum-deciders']
    count = 0
    for k, v in shards.items():
        members = v['committee-members']
        for i in members:
            if not i['is-harmony-slot']:
                count += 1
                stake = float(i['effective-stake'])
                bls_key = i['bls-public-key']
                addr = i['earning-account']
                if stake > upper or stake < lower:
                    print("validator: ", addr, " bls public key: ", bls_key) 
                    print("effective stake is out of range. The effective stake is ", stake)
                    print("")
                flag = False
    print("total slots verified:", count)
    if flag:
        return True
    else:
        return False

In [5]:
M5_test()

median stake is 0.0
lower bond is 0.0
upper bond is 0.0
validator:  one1r5sx7e64gefzy6dzzgaurvf3wzzjeac4ya0u29  bls public key:  92942d2a7167e6d429896dd6b526bace7cb8027e35b2e0f08285883f73ba0da0c76ee3a376794837de88f5861bb3e580
effective stake is out of range. The effective stake is  1.4375e+24

validator:  one1fchs24cqnzyxsxzmulxsldru7zl3zahxwwyqnk  bls public key:  4433d90f6ae044f553ef9f8de95f1a96889e43352d6bbead3d75cd8c8306758a5296ae550388f4bb87c95c9a54e8790c
effective stake is out of range. The effective stake is  1.0625e+24

validator:  one1x08eyghh8uhldsg2vkmss6dzul4zamkmmkqeeh  bls public key:  8ca8e14dcc0f5751a51ff1b1f61cddc2e574f43eb123c1e5925bb79f34fad00433127a2b9e12069a6c93ff8ddfb97d88
effective stake is out of range. The effective stake is  1.25e+24

validator:  one1gpz989mxf6uzu4ctz4xra7xcrytk5pe2h2lnn9  bls public key:  bc4b8f427fdb890053d63b17a739c01365e1c57408a90f85e1c52f4dbac712029f71fdb174288573b7bfefc784056804
effective stake is out of range. The effective stake is  1.

False